# Part B: Data Ploting

In [ ]:
import pandas as pd
df_all = pd.read_csv('municipality.csv')

In [ ]:
## Initialize
Ristat = [i for i in range(1,21)]
Pistat = set(i for i in range(1,107))

df_plot = pd.DataFrame(columns=['Lat','Lon','Mw','reg_name','reg_istat','prov_name','prov_istat','munic_name'])
df_plot = df_plot.astype({'Lat':float,'Lon':float,'Mw':float,'reg_name':str,'reg_istat':int,'prov_name':str,'prov_istat':int,'munic_name':str})
Pistat = set()
Ristat = []
list_geo_data = []


## Filter with Provinces

In [ ]:
## list of provinces names to plot
provinces = ['L\'Aquila','Chieti','Teramo']

## populate df_plot & Pistat
for i in provinces:
    X = df_all[df_all['prov_name'].str.match(i)]
    df_plot = pd.concat([df_plot,X], ignore_index=True)
    Pistat.add(X['prov_istat'].iloc[0])
for P in Pistat:
    lineP = 'geojson-italy-master/geojson/limits_P_'+str(P)+'_municipalities.geojson'
    list_geo_data.append(lineP)

#list_geo_data

## Filter with Regions

In [ ]:
## list of region names to plot
provinces = ['Abruzzo','Lazio']

## populate df_plot & Ristat
for i in provinces:
    X = df_all[df_all['reg_name'].str.match(i)]
    df_plot = pd.concat([df_plot,X], ignore_index=True)
    Ristat.append(X['reg_istat'].iloc[0])
for R in Ristat:
    lineR = 'geojson-italy-master/geojson/limits_R_'+str(R)+'_municipalities.geojson'
    list_geo_data.append(lineR)

#list_geo_data

## Plot

In [ ]:
df_loc = df_plot[["Lat","Lon","Mw"]].copy() # Copy for later
df_plot = df_plot.groupby(['munic_name']).size().to_frame('Count').reset_index()

In [ ]:
## Generate colour map
from branca.colormap import linear

colormap = linear.YlOrRd_09.scale(
    df_plot.Count.min(),
    df_plot.Count.max())
colormap

## Create pandas series for choropleth
plot_dict = df_plot.set_index('munic_name')['Count'] # plot_dict index 'munic_name' links to 'name' inside municipality json files
#colormap

In [ ]:
import folium
import numpy as np
def get_color(feature):
    value = plot_dict.get(feature['properties']['name']) # feature[...] maps the index of plot_dict to its partner in json data file
    if value is None:
        return '#00000000' #00000000 -> Transparent
    else:
        return colormap(value)

Lat=42.349850
Lon=13.399509

m = folium.Map([Lat,Lon], zoom_start=8)#,tiles='Stamen Terrain')

for i in list_geo_data:
  folium.GeoJson(
    i,
    name='earthquake/'+str(i),
    style_function=lambda feature: {
      'fillColor':get_color(feature),
      'color': 'black',
      'weight': 1,
      'dashArray': '5, 5',
      'fillOpacity': 0.9,
    }
  ).add_to(m)

## Add map features
for i in range(len(df_loc)):
    folium.CircleMarker(
        location=[df_loc.Lat[i],df_loc.Lon[i]],
        radius=0.5,
        fill=True,
        popup=[format(np.float(df_loc.Lat[i]),'.2f'),format(np.float(df_loc.Lon[i]),'.2f'),format(df_loc.Mw[i],'.2f')],
    ).add_to(m)    
m.add_child(colormap)
folium.LayerControl().add_to(m)

## Show & Save
m.save('Maps/EarthquakeMap.html')
m